In [1]:
def pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, nstart=None, weight='weight', dangling=None):
    if len(G) == 0: 
        return {} 
  
    if not G.is_directed(): 
        D = G.to_directed() 
    else: 
        D = G
        
    # Create a copy in (right) stochastic form 
    W = nx.stochastic_graph(D, weight=weight) 
    N = W.number_of_nodes()
    
    # Choose fixed starting vector if not given 
    if nstart is None: 
        x = dict.fromkeys(W, 1.0 / N) 
    else: 
        # Normalized nstart vector 
        s = float(sum(nstart.values())) 
        x = dict((k, v / s) for k, v in nstart.items()) 
        
    if personalization is None:
        # Assign uniform personalization vector if not given 
        p = dict.fromkeys(W, 1.0 / N)
    else: 
        missing = set(G) - set(personalization) 
        if missing: 
            raise NetworkXError('Personalization dictionary must have a value for every node. Missing nodes %s' % missing) 
        s = float(sum(personalization.values())) 
        p = dict((k, v / s) for k, v in personalization.items())
        
    if dangling is None:
        # Use personalization vector if dangling vector not specified 
        dangling_weights = p 
    else: 
        missing = set(G) - set(dangling) 
        if missing: 
            raise NetworkXError('Dangling node dictionary must have a value for every node. Missing nodes %s' % missing) 
        s = float(sum(dangling.values())) 
        dangling_weights = dict((k, v/s) for k, v in dangling.items())
        
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
    
     # power iteration: make up to max_iter iterations 
    for _ in range(max_iter): 
        xlast = x 
        x = dict.fromkeys(xlast.keys(), 0) 
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes) 
        for n in x:
            # this matrix multiply looks odd because it is 
            # doing a left multiply x^T=xlast^T*W 
            for nbr in W[n]: 
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight] 
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n] 
  
        # check convergence, l1 norm 
        err = sum([abs(x[n] - xlast[n]) for n in x]) 
        if err < N*tol: 
            return x 
    raise NetworkXError('Pagerank: power iteration failed to converge in %d iterations.' % max_iter)

In [2]:
import networkx as nx

In [5]:
G = nx.barabasi_albert_graph(60, 41) 
pr = nx.pagerank(G, 0.4)

In [6]:
print(pr)

{0: 0.028052290070731803, 1: 0.012183016941013425, 2: 0.012570764524140425, 3: 0.012763545911401247, 4: 0.01254385932318987, 5: 0.012759652998990584, 6: 0.013580149000897337, 7: 0.01296869733144541, 8: 0.011397965305089377, 9: 0.01277907870947181, 10: 0.011381972094023467, 11: 0.013182524213139384, 12: 0.01317955393574873, 13: 0.012951334571320501, 14: 0.012943226790270492, 15: 0.013170885047003799, 16: 0.012963450658613972, 17: 0.013381888161762888, 18: 0.013567868533192126, 19: 0.01318370642719815, 20: 0.013370142598196372, 21: 0.01316906159548988, 22: 0.013162622451814869, 23: 0.012973318583988841, 24: 0.013373093877750583, 25: 0.013174393698354447, 26: 0.013170162559553361, 27: 0.013577110996987383, 28: 0.013161806963620282, 29: 0.012969398620647651, 30: 0.012987564119811673, 31: 0.013362044091829381, 32: 0.012977747651273736, 33: 0.01257262343356531, 34: 0.013172186976574202, 35: 0.0127651574249563, 36: 0.012764924081416613, 37: 0.012744778450884097, 38: 0.013778340925491907, 39: 